## Imports

In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
#from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

import tensorflow as tf
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from tensorflow.keras import layers, models



Tensorflow version 2.15.1


## Parameters

In [2]:

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 12
DATASET_DIR = "D:\projects\Amin_Daghig_carmodel_res50\model1_classes"

In [3]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices("GPU"))

TF version: 2.15.1
Built with CUDA: False
GPUs: []


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [5]:
!nvidia-smi

Tue Aug 19 11:12:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.97                 Driver Version: 580.97         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              4W /   35W |     552MiB /   4096MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#loading in rgb we can grayscale and do agumentation also

(val_ds2 , train_ds2) = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

Found 3037 files belonging to 12 classes.
Using 2430 files for training.
Using 607 files for validation.


In [5]:
import numpy as np

y_train = np.concatenate([labels.numpy() for _, labels in train_ds2], axis=0)


In [6]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

In [7]:
class_weights = dict(zip(np.unique(y_train), class_weights))
print(class_weights)

{0: 1.806547619047619, 1: 0.5214776632302406, 2: 0.6485042735042735, 3: 0.9367283950617284, 4: 0.6021825396825397, 5: 1.297008547008547, 6: 1.2645833333333334, 7: 2.529166666666667, 8: 1.5328282828282829, 9: 1.631720430107527, 10: 0.6569264069264069, 11: 1.9455128205128205}


In [8]:
'''
data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
])

'''


data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(factor=0.3)
])


In [15]:
from tensorflow.keras.applications import MobileNetV3Large

base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224,224,3))


# Freeze base model initially
for layer in base_model.layers:
    layer.trainable = False


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v3/weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5: None -- [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)

In [19]:
from tensorflow.keras.applications import MobileNetV3Large
# Build MobileNetV3 Large without the top classification layers
base_model = MobileNetV3Large(
    include_top=False,    # exclude the top layer
    weights=None,         # don't load weights from the internet
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'         # global average pooling at the end
)

# Load your local weights file here
weights_path = 'D:\projects\Amin_Daghig_carmodel_res50\weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5'  # update this path
base_model.load_weights(weights_path)

In [20]:
base_model.summary()

Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_7 (Rescaling)     (None, 224, 224, 3)          0         ['input_9[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 112, 112, 16)         432       ['rescaling_7[0][0]']         
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 112, 112, 16)         64        ['Conv[0][0]']                
 lization)                                                                         

In [22]:
# Freeze base model initially
for layer in base_model.layers:
    layer.trainable = False


In [24]:
inputs = base_model.input
x = data_augmentation(inputs)               # augmentation before EfficientNet
x = base_model(x, training=False)           # pass through EfficientNet
#x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(64, activation="relu")(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs, outputs)

In [ ]:
'''
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
predictions = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)
'''

In [25]:
model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [26]:
history = model.fit(   #no defined LR
    train_ds2,
    validation_data=val_ds2,
    epochs=10,
    verbose=1,
    class_weight=class_weights
)

Epoch 1/10


19/19 [==============================] - 44s 2s/step - loss: 2.7540 - accuracy: 0.1351 - val_loss: 2.3954 - val_accuracy: 0.1650
Epoch 2/10
19/19 [==============================] - 38s 2s/step - loss: 2.3412 - accuracy: 0.1993 - val_loss: 2.2475 - val_accuracy: 0.2317
Epoch 3/10
19/19 [==============================] - 37s 2s/step - loss: 2.1333 - accuracy: 0.2570 - val_loss: 2.1078 - val_accuracy: 0.3230
Epoch 4/10
19/19 [==============================] - 37s 2s/step - loss: 2.0175 - accuracy: 0.3180 - val_loss: 2.0123 - val_accuracy: 0.3502
Epoch 5/10
19/19 [==============================] - 37s 2s/step - loss: 1.8916 - accuracy: 0.3147 - val_loss: 1.9629 - val_accuracy: 0.3593
Epoch 6/10
19/19 [==============================] - 37s 2s/step - loss: 1.7688 - accuracy: 0.3542 - val_loss: 1.8084 - val_accuracy: 0.3868
Epoch 7/10
19/19 [==============================] - 38s 2s/step - loss: 1.7035 - accuracy: 0.4020 - val_loss: 1.7650 - val_accuracy: 0.4177
Epoch 8/10
19/19 [

In [27]:
#second stage trasining
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [28]:
history_finetune = model.fit(
    train_ds2,
    validation_data=val_ds2,
    epochs=12,
    verbose=1
)

Epoch 1/12
19/19 [==============================] - 43s 2s/step - loss: 1.4905 - accuracy: 0.4843 - val_loss: 1.4086 - val_accuracy: 0.5267
Epoch 2/12
19/19 [==============================] - 38s 2s/step - loss: 1.1876 - accuracy: 0.5881 - val_loss: 1.1937 - val_accuracy: 0.6140
Epoch 3/12
19/19 [==============================] - 38s 2s/step - loss: 0.8948 - accuracy: 0.7002 - val_loss: 1.1596 - val_accuracy: 0.6243
Epoch 4/12
19/19 [==============================] - 37s 2s/step - loss: 0.8589 - accuracy: 0.7150 - val_loss: 1.0073 - val_accuracy: 0.6930
Epoch 5/12
19/19 [==============================] - 37s 2s/step - loss: 0.7128 - accuracy: 0.7512 - val_loss: 1.0556 - val_accuracy: 0.6798
Epoch 6/12
19/19 [==============================] - 38s 2s/step - loss: 0.6329 - accuracy: 0.7908 - val_loss: 0.9911 - val_accuracy: 0.6996
Epoch 7/12
19/19 [==============================] - 38s 2s/step - loss: 0.5080 - accuracy: 0.8171 - val_loss: 0.9087 - val_accuracy: 0.7407
Epoch 8/12
19/19 [==

In [29]:
history_finetune = model.fit(
    train_ds2,
    validation_data=val_ds2,
    epochs=7,
    verbose=1
)

Epoch 1/7
19/19 [==============================] - 38s 2s/step - loss: 0.3004 - accuracy: 0.8929 - val_loss: 0.9465 - val_accuracy: 0.7465
Epoch 2/7
19/19 [==============================] - 38s 2s/step - loss: 0.2914 - accuracy: 0.9061 - val_loss: 0.8101 - val_accuracy: 0.7630
Epoch 3/7
19/19 [==============================] - 38s 2s/step - loss: 0.2366 - accuracy: 0.9390 - val_loss: 0.8821 - val_accuracy: 0.7634
Epoch 4/7
19/19 [==============================] - 38s 2s/step - loss: 0.2500 - accuracy: 0.9077 - val_loss: 0.9166 - val_accuracy: 0.7728
Epoch 5/7
19/19 [==============================] - 38s 2s/step - loss: 0.2823 - accuracy: 0.8946 - val_loss: 0.8307 - val_accuracy: 0.7728
Epoch 6/7
19/19 [==============================] - 38s 2s/step - loss: 0.1819 - accuracy: 0.9407 - val_loss: 1.1425 - val_accuracy: 0.7317
Epoch 7/7
19/19 [==============================] - 38s 2s/step - loss: 0.2857 - accuracy: 0.9077 - val_loss: 0.9203 - val_accuracy: 0.7473


In [30]:
model.save("moblinenetv3large_13class_v7.h5")

c:\anaconda\envs\yolov7-gpu-env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
